In [1]:
import ee
import geemap
import os
import shapely
import json
import mgrs
import pandas as pd
import ipywidgets
import folium
import geemap.eefolium as gmap
import traceback

# Initialize Map Object

In [4]:
ee.Initialize()

In [5]:
Map = gmap.Map()

# Import Polygons

In [6]:
def import_aois(csv_loc):    

        df_labels = pd.read_csv(csv_loc)
        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

        polygons = []
        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])


        tiles = []
        tiles_dic = {}
        polygon_id = 0 
        coordinates = []
        m = mgrs.MGRS()
        for items in polygons:
            polygon_id += 1 
            for item in items:
                for lon_lat in item:
                    coordinates.append(lon_lat)
                    c = m.toMGRS(lon_lat[1], lon_lat[0])
                    tile = c[0:5]
                    

                    if polygon_id in tiles_dic:

                        tiles_dic[polygon_id].append(tile)

                    else:

                        tiles_dic[polygon_id] = [tile]

                    tiles.append(tile)

                tiles_dic[polygon_id] = list(set(tiles_dic[polygon_id]))

        tiles = list(set(tiles))

        df_labels["tiles"] = tiles_dic.values()

        #bounding box

        min_lon = min([i[0] for i in coordinates])
        min_lat = min([i[1] for i in coordinates])
        max_lon = max([i[0] for i in coordinates])
        max_lat = max([i[1] for i in coordinates])

        bounding_box = min_lon,min_lat,max_lon,max_lat
        
        return polygons

In [7]:
polygons = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

/Users/purgatorid/opt/anaconda3/envs/gee-conda/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [8]:
polygons[0]

[[[9.088783, 5.753152],
  [9.179249, 5.703106],
  [9.248257, 5.792946],
  [9.236069, 5.845887],
  [9.090672, 5.844009],
  [9.088783, 5.753152]]]

In [9]:
Map = gmap.Map()

# start and end of time series
startDate = ee.Date('2020-01-01')
stopDate  = ee.Date('2020-12-31')
TCI_RGB = ['TCI_R', 'TCI_G', 'TCI_B']
count_loop = 0
count_bad = 0  
good_images = []
good_dict = {}
feature_id = 0 
features = []


for poly in polygons[0:1]:
#     Map = gmap.Map()
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)
    
    centroid = polys.centroid()
    lng, lat = centroid.getInfo()['coordinates']
    print("lng = {}, lat = {}".format(lng, lat))
    Map.setCenter(lng, lat, 12)

    # Add Earth Engine dataset
    # Sentinel 2 L2A ImageCollection
    # filter by bounds and data
    collection = ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterBounds(polys)\
        .filterDate(startDate,stopDate)\
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
        .sort('CLOUD_COVER',True)\
        .select(TCI_RGB)
    
    # viz params
    vis = {'bands': TCI_RGB, "gamma": [2, 2, 2]}

    # use only first image from collection
    img = ee.Image(collection.first())
    Map.addLayer(img, vis)
    
    # clip image to the specific geometry
    img = img.clip(polys)

    # use the mean image - would require clipping image again
#     collection_mean = collection.mean()


    # print out the number of images in the ImageCollection
    count = collection.size().getInfo()
    print("Count: ", count)
    
    
    
    count_loop += 1
    
    # if more than zero in the image collection
    if count > 0:
#         Map.addLayer(img,vis, f"Image ID {count_loop}")
        # Map.addLayer(collection,vis,"Original Collection")
        # Map.addLayer(collection_mean,vis, "Collection Mean")
        good_images.append(img)
        print(f"Added Image ID {count_loop}")
    
    else:
        count_bad += 1 
        continue

    

print(f'found {count_bad} polygons without a corresponding image collection')


# Create custom collection with list of images for DL
custom_collection = ee.ImageCollection.fromImages(good_images)
Map.addLayer(custom_collection, vis)
image_dl_list = custom_collection.toList(105)

# add all polygon features to map

fc = ee.FeatureCollection(features)
Map.add_layer(fc, {}, 'default display')

Map

lng = 9.163954053102769, lat = 5.7876496002257225
Count:  7
Added Image ID 1
found 0 polygons without a corresponding image collection


In [16]:
RGB = ['TCI_R', 'TCI_G', 'TCI_B']

# Download the specific image
# for image_obj in range(len(good_images)):
for i in range(len(good_images),5):
    downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'image3',"driveFileNamePrefix":str(i)}  # scale means resolution.
    image_to_dl = ee.Image(image_dl_list.get(i))
    # img_2 = image_to_dl.select('B.+')
    img_2 = image_to_dl.select(RGB)
    name = img_2.getInfo()["id"].split("/")[-1]
    print("Image to Download:",name)
    task = ee.batch.Export.image(img_2, name, downConfig)
    task.start()

while task.status()["state"] == 'RUNNING':
    print(task.status(), end="\r", flush=True)

Image to Download: 20200105T094309_20200105T095703_T32NNM
Image to Download: 20200105T094309_20200105T095703_T32NMM
Image to Download: 20200104T092401_20200104T093437_T33NTG
Image to Download: 20200104T092401_20200104T093437_T33NVF
Image to Download: 20200104T092401_20200104T093437_T33NTF
Image to Download: 20200105T094309_20200105T095703_T32NML
Image to Download: 20200104T092401_20200104T093437_T33NUF
Image to Download: 20200104T092401_20200104T093437_T33NVF
Image to Download: 20200107T093351_20200107T094442_T32NRL
Image to Download: 20200107T093351_20200107T094442_T32NRL
Image to Download: 20200114T092331_20200114T093418_T33NVE
Image to Download: 20200105T094309_20200105T095703_T32NML
Image to Download: 20200114T092331_20200114T093418_T33NVE
Image to Download: 20200213T092101_20200213T093321_T32NRK
Image to Download: 20200205T091049_20200205T091719_T33NWE
Image to Download: 20200205T091049_20200205T091719_T33NWE
Image to Download: 20200205T091049_20200205T091719_T33NXE
Image to Downl

# Sandbox

In [37]:
ee.Initialize()

In [3]:
def import_aois(csv_loc):    

        df_labels = pd.read_csv(csv_loc)
        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

        polygons = []
        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])


        tiles = []
        tiles_dic = {}
        polygon_id = 0 
        coordinates = []
        m = mgrs.MGRS()
        for items in polygons:
            polygon_id += 1 
            for item in items:
                for lon_lat in item:
                    coordinates.append(lon_lat)
                    c = m.toMGRS(lon_lat[1], lon_lat[0])
                    tile = c[0:5]
                    

                    if polygon_id in tiles_dic:

                        tiles_dic[polygon_id].append(tile)

                    else:

                        tiles_dic[polygon_id] = [tile]

                    tiles.append(tile)

                tiles_dic[polygon_id] = list(set(tiles_dic[polygon_id]))

        tiles = list(set(tiles))

#         df_labels["tiles"] = tiles_dic.values()

        #bounding box

        min_lon = min([i[0] for i in coordinates])
        min_lat = min([i[1] for i in coordinates])
        max_lon = max([i[0] for i in coordinates])
        max_lat = max([i[1] for i in coordinates])

        bounding_box = min_lon,min_lat,max_lon,max_lat
        
        return polygons[0:1]

In [4]:
polygons_path = 'D:/canopy_data/csvs/polygons_101320.csv'

polygons = import_aois(polygons_path)

In [47]:
polygons

[[[[9.088783, 5.753152],
   [9.179249, 5.703106],
   [9.248257, 5.792946],
   [9.236069, 5.845887],
   [9.090672, 5.844009],
   [9.088783, 5.753152]]]]

In [7]:
# Map = gmap.Map()

# start and end of time series
startDate = ee.Date('2020-01-01')
stopDate  = ee.Date('2020-12-31')
TCI_RGB = ['TCI_R', 'TCI_G', 'TCI_B']
count_loop = 0
count_bad = 0  
good_images = []
good_dict = {}
feature_id = 0 
features = []
img_objs = []


for poly in polygons[0:1]:
#     Map = gmap.Map()
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)
    
    centroid = polys.centroid()
    lng, lat = centroid.getInfo()['coordinates']
    print("lng = {}, lat = {}".format(lng, lat))
    Map.setCenter(lng, lat, 12)
    
    
    sensor_list = ['COPERNICUS/S2','COPERNICUS/S2_SR']
    for sensor in sensor_list:
        # Add Earth Engine dataset
        # Sentinel 2 L2A ImageCollection
        # filter by bounds and data
        collection = ee.ImageCollection(sensor)\
            .filterBounds(polys)\
            .filterDate(startDate,stopDate)\
            .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
            .sort('CLOUD_COVER',True)


        # use only first image from collection
        img = ee.Image(collection.first())

        # clip image to the specific geometry
        img = img.clip(polys)
        img_objs.append(img)

lng = 9.163954053102769, lat = 5.7876496002257225


In [8]:
for img in img_objs:
    dic = img.getInfo()
    print(json.dumps(str(dic)))
#     product_id = str(img.getInfo()["properties"]['PRODUCT_ID'])
#     print(product_id)
#     print([product_id.split("_")[1],product_id.split("_")[2], product_id.split("_")[-2]])
    

"{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [295, 264], 'origin': [164, 897], 'crs': 'EPSG:32632', 'crs_transform': [60, 0, 499980, 0, -60, 700020]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1767, 1580], 'origin': [984, 5384], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 499980, 0, -10, 700020]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1767, 1580], 'origin': [984, 5384], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 499980, 0, -10, 700020]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1767, 1580], 'origin': [984, 5384], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 499980, 0, -10, 700020]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [88

In [57]:
mosaic = custom_collection.mosaic().clip(polys)

In [60]:
RGB = ['TCI_R', 'TCI_G', 'TCI_B']

# Download the specific image
# for image_obj in range(len(good_images)):
downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'image3',"driveFileNamePrefix":"mosaic"}  # scale means resolution.
# image_to_dl = ee.Image(image_dl_list.get(i))
# img_2 = image_to_dl.select('B.+')
# img_2 = image_to_dl.select(RGB)
name = "mosaic"
print("Image to Download:",name)
task = ee.batch.Export.image(mosaic, name, downConfig)
task.start()

while task.status()["state"] == 'RUNNING':
    print(task.status(), end="\r", flush=True)

Image to Download: mosaic


In [63]:
while task.status()["state"] == 'RUNNING' or task.status()["state"] == 'READY':
    print(task.status(), end="\r", flush=True)

In [9]:
len(img_objs)

2

In [12]:
sys_ind_1C = img_objs[0].getInfo()['properties']['system:index']
sys_ind_2A = img_objs[1].getInfo()['properties']['system:index']

In [13]:
assert sys_ind_1C == sys_ind_2A

In [14]:
sys_ind_1C

'20200105T094309_20200105T095703_T32NNM'

In [15]:
img_objs[1].getInfo()['properties']

{'system:footprint': {'type': 'Polygon',
  'coordinates': [[[9.088783, 5.753151999999999],
    [9.179248999999999, 5.703106],
    [9.248257000000002, 5.792946],
    [9.236069, 5.845887000000001],
    [9.090672, 5.844009],
    [9.088783, 5.753151999999999]]]},
 'DATATAKE_IDENTIFIER': 'GS2B_20200105T094309_014791_N02.13',
 'AOT_RETRIEVAL_ACCURACY': 0,
 'SPACECRAFT_NAME': 'Sentinel-2B',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE': 0,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 283.354926056,
 'CLOUD_SHADOW_PERCENTAGE': 0.063872,
 'MEAN_SOLAR_AZIMUTH_ANGLE': 143.643657178,
 'VEGETATION_PERCENTAGE': 90.235651,
 'SOLAR_IRRADIANCE_B12': 87.75,
 'SOLAR_IRRADIANCE_B10': 365.41,
 'SENSOR_QUALITY': 'PASSED',
 'SOLAR_IRRADIANCE_B11': 247.08,
 'GENERATION_TIME': 1578225361000,
 'SOLAR_IRRADIANCE_B8A': 953.93,
 'FORMAT_CORRECTNESS': 'PASSED',
 'CLOUD_COVERAGE_ASSESSMENT': 1.778067,
 'THIN_CIRRUS_PERCENTAGE': 1.702274,
 'system:time_end': 1578218343367,
 'WATER_VAPOUR_RETRIEVAL_ACCURACY': 0,
 'system:time_start

In [104]:
img_objs[1].getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [295, 264],
   'origin': [164, 897],
   'crs': 'EPSG:32632',
   'crs_transform': [60, 0, 499980, 0, -60, 700020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1767, 1580],
   'origin': [984, 5384],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1767, 1580],
   'origin': [984, 5384],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1767, 1580],
   'origin': [984, 5384],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020

In [17]:
img_id = img_objs[1].getInfo()['id']

img_id

'COPERNICUS/S2_SR/20200105T094309_20200105T095703_T32NNM'

In [21]:
l1c_img_id = img_id.replace('_SR', '')
l1c_img_id

'COPERNICUS/S2/20200105T094309_20200105T095703_T32NNM'

In [22]:
new_image = ee.Image(l1c_img_id)
new_image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32632',
   'crs_transform': [60, 0, 499980, 0, -60, 700020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [24]:
assert new_image.getInfo()['properties']['system:index'] == sys_ind_1C

In [26]:
help(new_image.select)

Help on method select in module ee.image:

select(opt_selectors=None, opt_names=None, *args) method of ee.image.Image instance
    Selects bands from an image.
    
    Can be called in one of two ways:
      - Passed any number of non-list arguments. All of these will be
        interpreted as band selectors. These can be band names, regexes, or
        numeric indices. E.g.
        selected = image.select('a', 'b', 3, 'd');
      - Passed two lists. The first will be used as band selectors and the
        second as new names for the selected bands. The number of new names
        must match the number of selected bands. E.g.
        selected = image.select(['a', 4], ['newA', 'newB']);
    
    Args:
      opt_selectors: An array of names, regexes or numeric indices specifying
          the bands to select.
      opt_names: An array of strings specifying the new names for the
          selected bands.
      *args: Selector elements as varargs.
    
    Returns:
      An image with the

In [27]:
B10 = new_image.select('B10')
B10.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B10',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32632',
   'crs_transform': [60, 0, 499980, 0, -60, 700020]}],
 'id': 'COPERNICUS/S2/20200105T094309_20200105T095703_T32NNM',
 'version': 1578292702865577,
 'properties': {'DATATAKE_IDENTIFIER': 'GS2B_20200105T094309_014791_N02.08',
  'SPACECRAFT_NAME': 'Sentinel-2B',
  'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 283.354926056,
  'MEAN_SOLAR_AZIMUTH_ANGLE': 143.643657178,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[8.99972909124152, 5.339931457424325],
    [8.99979897450319, 5.3398414720050305],
    [9.855998550887362, 5.339244984457492],
    [9.856220872753056, 5.339467211589392],
    [9.856409207934037, 5.340031100096717],
    [9.856954198273879, 5.34184268076418],
    [9.857499362442397, 5.344015374998806],
    [9.8654884593693, 5.377473626412896],
    [9.924376093292087, 5.63934020726525

In [28]:
img.getInfo()['bands']

[{'id': 'B1',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [295, 264],
  'origin': [164, 897],
  'crs': 'EPSG:32632',
  'crs_transform': [60, 0, 499980, 0, -60, 700020]},
 {'id': 'B2',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [1767, 1580],
  'origin': [984, 5384],
  'crs': 'EPSG:32632',
  'crs_transform': [10, 0, 499980, 0, -10, 700020]},
 {'id': 'B3',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [1767, 1580],
  'origin': [984, 5384],
  'crs': 'EPSG:32632',
  'crs_transform': [10, 0, 499980, 0, -10, 700020]},
 {'id': 'B4',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [1767, 1580],
  'origin': [984, 5384],
  'crs': 'EPSG:32632',
  'crs_transform': [10, 0, 499980, 0, -10, 700020]},
 {'id': 'B5',
  'data_type': {'type': 'PixelType',
   'prec

In [29]:
img2 = img.addBands(B10)

img2.getInfo()['bands']

[{'id': 'B1',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [295, 264],
  'origin': [164, 897],
  'crs': 'EPSG:32632',
  'crs_transform': [60, 0, 499980, 0, -60, 700020]},
 {'id': 'B2',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [1767, 1580],
  'origin': [984, 5384],
  'crs': 'EPSG:32632',
  'crs_transform': [10, 0, 499980, 0, -10, 700020]},
 {'id': 'B3',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [1767, 1580],
  'origin': [984, 5384],
  'crs': 'EPSG:32632',
  'crs_transform': [10, 0, 499980, 0, -10, 700020]},
 {'id': 'B4',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [1767, 1580],
  'origin': [984, 5384],
  'crs': 'EPSG:32632',
  'crs_transform': [10, 0, 499980, 0, -10, 700020]},
 {'id': 'B5',
  'data_type': {'type': 'PixelType',
   'prec

In [42]:
img.get('system:index').getInfo()

'20200105T094309_20200105T095703_T32NNM'

In [44]:
ee_img_ind = img.get('system:index')

In [45]:
ee_img_ind

In [50]:
front = ee.String('COPERNICUS/S2/')

front

In [51]:
full = front.cat(ee_img_ind)

full

In [52]:
full.getInfo()

'COPERNICUS/S2/20200105T094309_20200105T095703_T32NNM'

In [54]:
help(ee.Image)

Help on class Image in module ee.image:

class Image(ee.element.Element)
 |  Image(*args, **kwargs)
 |  
 |  An object to represent an Earth Engine image.
 |  
 |  Method resolution order:
 |      Image
 |      ee.element.Element
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  And = Image.and(*args, **kwargs)
 |      Returns 1 iff both values are non-zero for each matched pair of bands in
 |      image1 and image2. If either image1 or image2 has only 1 band, then it is
 |      used against all the bands in the other image. If the images have the same
 |      number of bands, but not the same names, they're used pairwise in the
 |      natural order. The output bands are named for the longer of the two inputs,
 |      or if they're equal in length, in image1's order. The type of the output
 |      pixels is boolean.
 |      
 |      Args:
 |        image1: The image from which the left operand bands 

In [55]:
help(ee.computedobject.ComputedObject)

Help on class ComputedObject in module ee.computedobject:

class ComputedObject(ee.encodable.Encodable)
 |  ComputedObject(*args, **kwargs)
 |  
 |  A representation of an Earth Engine computed object.
 |  
 |  This is a base class for most API objects.
 |  
 |  The class itself is not abstract as it is used to wrap the return values of
 |  algorithms that produce unrecognized types with the minimal functionality
 |  necessary to interact well with the rest of the API.
 |  
 |  ComputedObjects come in two flavors:
 |  1. If func != null and args != null, the ComputedObject is encoded as an
 |     invocation of func with args.
 |  2. If func == null and args == null, the ComputedObject is a variable
 |     reference. The variable name is stored in its varName member. Note that
 |     in this case, varName may still be null; this allows the name to be
 |     deterministically generated at a later time. This is used to generate
 |     deterministic variable names for mapped functions, ens

In [57]:
help(full)

Help on String in module ee.ee_string object:

class String(ee.computedobject.ComputedObject)
 |  String(*args, **kwargs)
 |  
 |  An object to represent strings.
 |  
 |  Method resolution order:
 |      String
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, string)
 |      Construct a string wrapper.
 |      
 |      This constructor accepts the following args:
 |        1) A bare string.
 |        2) A ComputedObject returning a string.
 |      
 |      Args:
 |        string: The string to wrap.
 |  
 |  cat = String.cat(*args, **kwargs)
 |      Concatenates two strings.
 |      
 |      Args:
 |        string1: The first string.
 |        string2: The second string.
 |  
 |  compareTo = String.compareTo(*args, **kwargs)
 |      Compares two strings lexicographically. Returns: the value 0 if the two
 |      strings are lexicographically equal; a value less than 0 if string1 is les

In [58]:
full_encoded = full.encode()

full_encoded

TypeError: 'NoneType' object is not callable

In [30]:
def inject_B10(img):
    img_id = img.getInfo()['id']
    if '_SR' not in img_id:
        raise ValueError('Input image must be an L2A image')
        
    L1C_img_id = img_id.replace('_SR', '')
    L1C_img = ee.Image(L1C_img_id)
    
    B10 = L1C_img.select('B10')
    
    return img.addBands(B10)

In [35]:
sensor = sensor_list[1]

collection = ee.ImageCollection(sensor)\
    .filterBounds(polys)\
    .filterDate(startDate,stopDate)\
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
    .sort('CLOUD_COVER',True)\
    .map(inject_B10, opt_dropNulls=True)

EEException: ValueNode is empty

In [34]:
help(ee.ImageCollection.map)

Help on function map in module ee.collection:

map(self, algorithm, opt_dropNulls=None)
    Maps an algorithm over a collection.
    
    Args:
      algorithm: The operation to map over the images or features of the
          collection, a Python function that receives an image or features and
          returns one. The function is called only once and the result is
          captured as a description, so it cannot perform imperative operations
          or rely on external state.
      opt_dropNulls: If true, the mapped algorithm is allowed to return nulls,
          and the elements for which it returns nulls will be dropped.
    
    Returns:
      The mapped collection.
    
    Raises:
      ee_exception.EEException: if algorithm is not a function.



In [59]:
def map_test(img):
    coll = ee.ImageCollection('COPERNICUS/S2')
    
    return coll.first()

In [60]:
collection = ee.ImageCollection(sensor)\
    .filterBounds(polys)\
    .filterDate(startDate,stopDate)\
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
    .sort('CLOUD_COVER',True)\
    .map(map_test)

In [61]:
help(ee.ImageCollection.filterMetadata)

Help on function filterMetadata in module ee.collection:

filterMetadata(self, name, operator, value)
    Shortcut to add a metadata filter to a collection.
    
    This is equivalent to self.filter(Filter().metadata(...)).
    
    Args:
      name: Name of a property to filter.
      operator: Name of a comparison operator as defined
          by FilterCollection.  Possible values are: "equals", "less_than",
          "greater_than", "not_equals", "not_less_than", "not_greater_than",
          "starts_with", "ends_with", "not_starts_with", "not_ends_with",
          "contains", "not_contains".
      value: The value to compare against.
    
    Returns:
      The filtered collection.



In [63]:
ee_img_ind = img.get('system:index')

coll = ee.ImageCollection('COPERNICUS/S2')\
    .filterMetadata('system:index', 'equals', ee_img_ind)

In [64]:
dir(coll)

['And',
 'Or',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply_preparation_function',
 '_cast',
 '_getThumbURL',
 '_initialized',
 'aggregate_array',
 'aggregate_count',
 'aggregate_count_distinct',
 'aggregate_first',
 'aggregate_histogram',
 'aggregate_max',
 'aggregate_mean',
 'aggregate_min',
 'aggregate_product',
 'aggregate_sample_sd',
 'aggregate_sample_var',
 'aggregate_stats',
 'aggregate_sum',
 'aggregate_total_sd',
 'aggregate_total_var',
 'args',
 'aside',
 'cache',
 'cast',
 'combine',
 'copyProperties',
 'count',
 'distance',
 'distinct',
 'draw',
 'elementType',
 'encode',
 'encode_cloud_value',
 'errorMatrix',
 'filter',
 'filterBounds',
 'filte

In [66]:
coll.size().getInfo()

1

In [67]:
coll.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32632',
   'crs_transform': [60, 0, 499980, 0, -60, 700020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [68]:
ee_img_ind.getInfo()

'20200105T094309_20200105T095703_T32NNM'

In [69]:
def inject_B10(img):
    ee_img_ind = img.get('system:index')
    coll = ee.ImageCollection('COPERNICUS/S2')\
        .filterMetadata('system:index', 'equals', ee_img_ind)
        
    L1C_img = coll.first()
    
    B10 = L1C_img.select('B10')
    
    return img.addBands(B10)

In [70]:
sensor = sensor_list[1]

collection = ee.ImageCollection(sensor)\
    .filterBounds(polys)\
    .filterDate(startDate,stopDate)\
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
    .sort('CLOUD_COVER',True)\
    .map(inject_B10, opt_dropNulls=True)

In [71]:
collection.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32632',
   'crs_transform': [60, 0, 499980, 0, -60, 700020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [72]:
collection = ee.ImageCollection(sensor)\
    .filterBounds(polys)\
    .filterDate(startDate,stopDate)\
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
    .sort('CLOUD_COVER',True)\
    .map(inject_B10, opt_dropNulls=True)\
    .map(map_test)